# GRT CNEOS Database Analysis

In this notebook we analyse the CNEOS database

In [1]:
from gravray import *
from gravray.util import *
from gravray.stats import *
from gravray.spice import *
from gravray.plot import *
from gravray.sampling import *
from gravray.orbit import *

from tqdm import tqdm

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%matplotlib nbagg

## Initialize

In [8]:
Spice.loadKernels()
earth=Body("EARTH")

## Load database

In [95]:
data=pd.read_csv("data/cneos_fireball_data.csv")
#cond=data["Calculated Total Impact Energy (kt)"]>=0
#datadir=data[cond].dropna().sort_values(by="Calculated Total Impact Energy (kt)",ascending=[False]).reset_index(drop=True)
datadir=data.dropna(subset=["Latitude (deg.)"]).sort_values(by="Calculated Total Impact Energy (kt)",ascending=[False]).reset_index(drop=True)
print("Number of events:",len(datadir))
datadir.head(5)

Number of events: 609


,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2013-02-15 03:20:33,54.8N,61.1E,23.3,18.6,12.8,-13.3,-2.4,3.750000e+14,440.0
1,2018-12-18 23:48:20,56.9N,172.4E,25.6,32.0,6.3,-3.0,-31.2,1.300000e+14,173.0
2,2010-12-25 23:24:00,38.0N,158.0E,26.0,18.1,18.0,-2.0,-4.0,2.000000e+13,33.0
3,2009-10-08 02:57:00,4.2S,120.6E,19.1,19.2,14.0,-16.0,-6.0,2.000000e+13,33.0
4,1994-02-01 22:38:09,2.7N,164.1E,NaN,NaN,NaN,NaN,NaN,1.820000e+13,30.0


In [112]:
datadir["Aloc"]=0
datadir["hloc"]=0
datadir["vimp"]=0
datadir["Name"]=""
for i,ind in enumerate(tqdm(datadir.index)):
    bolide=datadir.loc[ind]
    dateandtime=datetime.strptime(bolide["Peak Brightness Date/Time (UT)"],
                                  "%Y-%m-%d %H:%M:%S").strftime("%m/%d/%Y %H:%M:%S UTC")
    datename=datetime.strptime(bolide["Peak Brightness Date/Time (UT)"],
                                "%Y-%m-%d %H:%M:%S").strftime("CNEOS-%Y-%m-%d")

    #Position
    lat=bolide["Latitude (deg.)"]
    lon=bolide["Longitude (deg.)"]
    alt=bolide["Altitude (km)"]
    
    if "N" in lat:
        lat=+float(lat.replace("N",""))
    elif "S" in lat:
        lat=-float(lat.replace("S",""))
    if "E" in lon:
        lon=+float(lon.replace("E",""))
    elif "W" in lon:
        lon=-float(lon.replace("W",""))
    datadir.loc[ind,"lat"]=lat
    datadir.loc[ind,"lon"]=lon
           
    if np.isnan(alt):
        alt=33.33
        datadir.loc[ind,"Altitude (km)"]=alt

    if np.isnan(bolide["Velocity (km/s)"]):
        datadir.loc[ind,"Velocity (km/s)"]=-1

    #Location
    location=Location(earth,lon*Angle.Deg,lat*Angle.Deg,alt*Const.km)
    
    #Convert speed into Aloc, hloc, vimp
    if np.isnan(bolide["vx"]):
        bolide[["vx","vy","vz"]]=[0,0,0]
        datadir.loc[ind,["vx","vy","vz"]]=[0,0,0]
        
    vbod=-np.array(bolide[["vx","vy","vz"]].values).astype("float64")
    
    Aloc,hloc,vimp=location.vbod2loc(vbod)

    datadir.loc[ind,"Ephemeris Date"]=dateandtime
    datadir.loc[ind,"Name"]=datename
    datadir.loc[ind,"Aloc"]=Aloc*Angle.Rad
    datadir.loc[ind,"hloc"]=hloc*Angle.Rad
    datadir.loc[ind,"vimp"]=vimp

100%|██████████| 609/609 [00:04<00:00, 140.48it/s]


In [113]:
datadir.to_csv("data/cneos_fireball_data_location.csv",index=False)

In [108]:
datadir.head(5)

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt),Aloc,hloc,vimp,Name,Ephemeris Date
0,2013-02-15 03:20:33,54.8N,61.1E,23.30,18.6,12.8,-13.3,-2.4,3.750000e+14,440.0,99.896113,15.924142,-18.614242,CNEOS-2013-02-15,02/15/2013 03:20:33 UTC
1,2018-12-18 23:48:20,56.9N,172.4E,25.60,32.0,6.3,-3.0,-31.2,1.300000e+14,173.0,349.433476,68.586703,-31.970768,CNEOS-2018-12-18,12/18/2018 23:48:20 UTC
2,2010-12-25 23:24:00,38.0N,158.0E,26.00,18.1,18.0,-2.0,-4.0,2.000000e+13,33.0,147.196052,60.888747,-18.547237,CNEOS-2010-12-25,12/25/2010 23:24:00 UTC
3,2009-10-08 02:57:00,4.2S,120.6E,19.10,19.2,14.0,-16.0,-6.0,2.000000e+13,33.0,27.463598,67.457494,-22.090722,CNEOS-2009-10-08,10/08/2009 02:57:00 UTC
4,1994-02-01 22:38:09,2.7N,164.1E,33.33,-1.0,0.0,0.0,0.0,1.820000e+13,30.0,0.000000,0.000000,-0.000000,CNEOS-1994-02-01,02/01/1994 22:38:09 UTC
